In [ ]:
class Image_Augmentation():
    def __init__(self, image_list, save_dir):
        self.image_list = image_list

    # Crop 기능 수행
    def crop(self):
        pass

    # Resize 기능 수행
    def resize(self):
        pass

    # Flip 기능 수행
    def flip(self):
        pass

    # Rotate 기능 수행
    def rotate(self):
        pass

    # Random Erase 기능 수헹
    def random_erase(self):
        pass

    # 처리 전, 후 이미지 확인 기능 수행
    def show_images(self, images):
        pass